In [2]:
import os
import requests
import pandas as pd
import time

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"
division = "SILVER"

print(api_key)

RGAPI-59d3c7c7-2a2d-49d6-a12e-cf0bd2614069


In [3]:
def getLadder(division, tier) -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/entries/{solo_duo}/{division}/{tier}?page=1&api_key={api_key}"
	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()

			return pd.DataFrame(data)
	
def getMasterLadder(type) -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/{type}/by-queue/{solo_duo}?api_key={api_key}"

	response = requests.get(url)

	print(response)

	if response.status_code == 200:
			data = response.json()
			players = data['entries']

			return pd.DataFrame(players)

def getMatchIDs(puuid):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20&api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()

			return data
	else:
		print(response)

In [4]:
ladder_1 = getLadder(division, "I")
ladder_2 = getLadder(division, "II")
ladder_3 = getLadder(division, "III")
ladder_4 = getLadder(division, "IV")

ladder = pd.concat([ladder_1, ladder_2, ladder_3, ladder_4])

ladder.shape

(820, 13)

In [ ]:
match_ids = []

try:
	for index, row in ladder.iterrows():
		matches = getMatchIDs(row.puuid)
		match_ids.extend(matches)

		if (index + 1) % 20 == 0:
			print(len(match_ids))
			time.sleep(30) # Sleep 30 seconds so that it won't hit the rate limit

	print(len(match_ids)) # should be 6000


except Exception as e: print(e)
finally:
	df = pd.DataFrame({ 'match_id': match_ids })

	print(df.shape)

	df.to_csv(f"./MatchIDs-{division}.csv")

395
786
1186
1586
1986
2386
2786
3177
3577
3977
4470
4851
5243
5637
6022
6380
6760
7160
7560
7960
8460
8858
9246
9643
10043
10443
10839
11239
11626
12026
12526
12926
13318
13718
14105
14505
14900
15300
15692
16092
16182
(16182, 1)


: 